In [166]:
import numpy as np
import pandas as pd
import re

In [167]:
pd.set_option('display.max_colwidth',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [168]:
df=pd.read_csv('gurgaon_properties_v2.csv')

In [169]:
df.head()
#focus ---> areawithType,additionalRoom,agePossesion,furnishDetails,features

,property_type,society,sector,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,floorNum,facing,agePossession,nearbyLocations,furnishDetails,features
0,house,NaN,sector 41,7.25,29293.0,2475.0,Plot area 275(229.94 sq.m.),9.0,8.0,3,servant room,4.0,South-West,10+ Year Old,"['Huda city centre metro station', 'Axis bank ATM', 'Hdfc ATM', 'Axis bank ATM', 'Citi bank ATM', 'Icici ATM', 'Punjab national bank ATM', 'Hdfc bank ATM', 'Standard chartered ATM', 'Centre For Sight Gurgaon Sector 29', 'Dr. Naval Mendiratta', 'Fortis Memorial Research Institute Fortis Vivekanand Hospital', 'Dr. Kutbuddin Akbary', 'Max Hospital', 'Shivam Hospital Gurgaon', 'Gardian Pharmacy', 'City Medical', 'Bharat petroleum', 'Hdfc bank', 'Icici bank', 'Beer & Whisky Bar', 'MoB Ministory of Beer', ""McDonald's"", 'Park Baluchi', ""Domino's Pizza"", 'Zura', 'The Oriental Bloom Chinese and Thai', 'Gola Sizzles', ""Hops 'N' Brew"", 'Pizza Hut', 'Spaghetti Kitchen & Bar', 'Bikanervala', 'Tocpao', 'PWO house', 'Black Mambaa', 'Gravity Space Bar', 'Roots', 'KFC', 'Ardor 29', 'ADDA', 'Gung the palace Korean restaurant', 'Walking Street', 'distillery', 'Swagath', 'Dhabba', 'Pizza Hut', 'Balaji Vegetarian Paradise', 'School of Inspired Leadership SOIL', 'Ncr library']","['5 Fan', '1 Exhaust Fan', '8 Geyser', '12 Light', '4 AC', '2 TV', '1 Modular Kitchen', '1 Curtains', '2 Bed', '2 Wardrobe', '1 Washing Machine', 'No Chimney', 'No Dining Table', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No Water Purifier']","['Security / Fire Alarm', 'Private Garden / Terrace', 'Water purifier', 'Maintenance Staff', 'Water Storage', 'No open drainage around', 'Visitor Parking', 'Park', 'Security Personnel', 'Natural Light', 'Internet/wi-fi connectivity', 'Airy Rooms', 'Spacious Interiors', 'Waste Disposal', 'Club house / Community Center']"
1,flat,vipul lavanya,sector 81,1.28,7174.0,1784.0,Carpet area: 1784 (165.74 sq.m.),3.0,3.0,3,"pooja room,servant room",1.0,North-East,5 to 10 Year Old,"['Yadav Clinic', 'Bangali Clinic', 'Dr. J. S. Sarkar Clinic', 'Orris Community Center', 'HP Petrol Pump', 'Vijay Petrol Pump', 'Petrol Pump', 'Essar Petrol Pump', 'Petrol pump Maitri motors', 'Petrol Pump Indian Oil', 'Indian Oil', ""McDonald's"", 'KFC', 'Pizza Hut', 'Rao Dhaba', ""McDonald's""]","['1 Bed', '3 Wardrobe', '7 Fan', '1 Exhaust Fan', '2 Geyser', '12 Light', '1 Modular Kitchen', '1 Chimney', '1 Curtains', 'No AC', 'No Dining Table', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Washing Machine', 'No Water Purifier']","['Feng Shui / Vaastu Compliant', 'Security / Fire Alarm', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Water Storage', 'Park', 'Visitor Parking']"
2,flat,m3m woodshire,sector 107,1.40,5929.0,2361.0,Super Built up area 2361(219.34 sq.m.),3.0,4.0,N,not available,1.0,East,0 to 1 Year Old,NaN,"['1 Light', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Modular Kitchen', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']","['Power Back-up', 'Intercom Facility', 'Lift(s)', 'Swimming Pool', 'Maintenance Staff', 'Park', 'Security Personnel', 'Internet/wi-fi connectivity', 'Shopping Centre', 'Fitness Centre / GYM', 'Club house / Community Center', 'Rain Water Harvesting']"
3,flat,ild greens,sector 37c,0.78,5714.0,1365.0,Super Built up area 1365(126.81 sq.m.)Built Up area: 1068.1 sq.ft. (99.23 sq.m.)Carpet area: 900 sq.ft. (83.61 sq.m.),2.0,2.0,3,not available,1.0,North,5 to 10 Year Old,"['The Esplanade Mall', 'Gurgaon Road', 'Delhi Jaipur Expressway', 'Green Field Public School', 'Sunrise University', 'K.D. Hospital', 'Indira Gandhi International Airport', 'Gurgaon Railway Station', 'infinity Business Park']","['2 Wardrobe', '3 Fan', '1 Exhaust Fan', '1 Geyser', '10 Light', '1 Modular Kitchen', '1 Chimney', 'No AC', 'No Bed', 'No Curtains', 'No Dining Table', 'No Microwave', 'No Fridge

In [170]:
df.duplicated().sum()

np.int64(121)

# **1.areaWithType**

In [171]:
df[['area','areaWithType']].sample(5)

,area,areaWithType
2205,1540.0,Super Built up area 1929(179.21 sq.m.)Built Up area: 1780 sq.ft. (165.37 sq.m.)Carpet area: 1540 sq.ft. (143.07 sq.m.)
3647,1800.0,Built Up area: 1800 (167.23 sq.m.)
3164,1350.0,Plot area 150(125.42 sq.m.)
1769,1640.0,Super Built up area 1640(152.36 sq.m.)
1730,1467.0,Plot area 163(136.29 sq.m.)


In [172]:
#This function extracts the Super Built up area
def get_super_built_up_area(text):
  if isinstance(text,str):
     match=re.search(r'Super Built up area (\d+\.?\d*)',text)
     if match:
        return float(match.group(1))

  return None

In [173]:
#This function extracts the Built Up area or carpet area
def get_area(text,area_type):
  if isinstance(text,str) and isinstance(area_type,str):
    match=re.search(area_type + r'\s*:\s*(\d+\.?\d*)',text)
    if match:
      return float(match.group(1))

  return None


In [174]:
#This function checks if the area is provided in sq.m and converts it to sqft if needed
def convert_to_sqft(text,area_value):

  if area_value is None:
    return None


  if isinstance(area_value,str):
      match=re.search(r'{}\((\d+\.?\d*)sq.m.\)'.format(area_value),text)
      if match:
        sq_m_value=float(match.group(1))
        return sq_m_value*10.7639

  return area_value

In [175]:
#Extract super built up area and convert it to sqft if needed
df['super_built_up_area']=df['areaWithType'].apply(get_super_built_up_area)
df['super_built_up_area']=df.apply(lambda x:convert_to_sqft(x['areaWithType'],x['super_built_up_area']),axis=1)

#Extract built up area and convert it to sqft if needed
df['built_up_area']=df['areaWithType'].apply(lambda x:get_area(x,'Built Up area'))
df['built_up_area']=df.apply(lambda x:convert_to_sqft(x['areaWithType'],x['built_up_area']),axis=1)

#Extract Carpet area and convert it to sqft if needed
df['carpet_area']=df['areaWithType'].apply(lambda x:get_area(x,'Carpet area'))
df['carpet_area']=df.apply(lambda x:convert_to_sqft(x['areaWithType'],x['carpet_area']),axis=1)


In [176]:
df[['area','areaWithType','super_built_up_area','built_up_area','carpet_area']].sample(5)

,area,areaWithType,super_built_up_area,built_up_area,carpet_area
1916,1423.0,Built Up area: 1423 (132.2 sq.m.)Carpet area: 1190 sq.ft. (110.55 sq.m.),NaN,1423.0,1190.0
925,1245.0,Super Built up area 1245(115.66 sq.m.),1245.0,NaN,NaN
2314,3200.0,Super Built up area 3200(297.29 sq.m.)Carpet area: 3156 sq.ft. (293.2 sq.m.),3200.0,NaN,3156.0
416,2400.0,Super Built up area 2475(229.94 sq.m.)Carpet area: 2400 sq.ft. (222.97 sq.m.),2475.0,NaN,2400.0
697,1120.0,Carpet area: 1120 (104.05 sq.m.),NaN,NaN,1120.0


In [177]:
a=df[~((df['super_built_up_area'].isnull()) | (df['built_up_area'].isnull()) | (df['carpet_area'].isnull()))][['area','areaWithType','super_built_up_area','built_up_area','carpet_area']]
a.head()

,area,areaWithType,super_built_up_area,built_up_area,carpet_area
3,1365.0,Super Built up area 1365(126.81 sq.m.)Built Up area: 1068.1 sq.ft. (99.23 sq.m.)Carpet area: 900 sq.ft. (83.61 sq.m.),1365.0,1068.10,900.00
4,1940.0,Super Built up area 1940(180.23 sq.m.)Built Up area: 1175.97 sq.ft. (109.25 sq.m.)Carpet area: 834.96 sq.ft. (77.57 sq.m.),1940.0,1175.97,834.96
6,1260.0,Super Built up area 1260(117.06 sq.m.)Built Up area: 1250 sq.ft. (116.13 sq.m.)Carpet area: 800 sq.ft. (74.32 sq.m.),1260.0,1250.00,800.00
23,1700.0,Super Built up area 1950(181.16 sq.m.)Built Up area: 1920 sq.ft. (178.37 sq.m.)Carpet area: 1700 sq.ft. (157.94 sq.m.),1950.0,1920.00,1700.00
27,2364.0,Super Built up area 2364(219.62 sq.m.)Built Up area: 2362 sq.ft. (219.44 sq.m.)Carpet area: 2360 sq.ft. (219.25 sq.m.),2364.0,2362.00,2360.00


In [178]:
a.shape

(530, 5)

In [179]:
df.shape

(3771, 20)

In [180]:
df[df['areaWithType'].str.contains('Plot',na=False)][['area','areaWithType','super_built_up_area','built_up_area','carpet_area']].head(5)

,area,areaWithType,super_built_up_area,built_up_area,carpet_area
0,2475.0,Plot area 275(229.94 sq.m.),NaN,NaN,NaN
8,1512.0,Plot area 168(140.47 sq.m.),NaN,NaN,NaN
11,3600.0,Plot area 400(334.45 sq.m.),NaN,NaN,NaN
16,NaN,Plot area 670(560.21 sq.m.),NaN,NaN,NaN
29,270.0,Plot area 270(25.08 sq.m.),NaN,NaN,NaN


In [181]:
df.isnull().sum()

,0
property_type,1
society,470
sector,0
price,19
price_per_sqft,19
area,19
areaWithType,1
bedRoom,1
bathroom,1
balcony,1


In [182]:
all_nan_df = df[((df['super_built_up_area'].isnull()) & (df['built_up_area'].isnull()) & (df['carpet_area'].isnull()))][['price','property_type','area','areaWithType','super_built_up_area','built_up_area','carpet_area']]

In [183]:
all_nan_df.head()

,price,property_type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
0,7.25,house,2475.0,Plot area 275(229.94 sq.m.),NaN,NaN,NaN
8,3.25,house,1512.0,Plot area 168(140.47 sq.m.),NaN,NaN,NaN
11,8.48,house,3600.0,Plot area 400(334.45 sq.m.),NaN,NaN,NaN
16,NaN,house,NaN,Plot area 670(560.21 sq.m.),NaN,NaN,NaN
29,0.34,house,270.0,Plot area 270(25.08 sq.m.),NaN,NaN,NaN


In [184]:
#function to extract plot area from areaWithType Column
def extract_plot_area(area_with_type):
  if isinstance(area_with_type,str):
    match=re.search(r'Plot area (\d+\.?\d*)',area_with_type)
    if match:
      return float(match.group(1))

  return None

In [185]:
all_nan_df['built_up_area']=all_nan_df['areaWithType'].apply(extract_plot_area)

In [186]:
all_nan_df

,price,property_type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
0,7.25,house,2475.0,Plot area 275(229.94 sq.m.),NaN,275.00,NaN
8,3.25,house,1512.0,Plot area 168(140.47 sq.m.),NaN,168.00,NaN
11,8.48,house,3600.0,Plot area 400(334.45 sq.m.),NaN,400.00,NaN
16,NaN,house,NaN,Plot area 670(560.21 sq.m.),NaN,670.00,NaN
29,0.34,house,270.0,Plot area 270(25.08 sq.m.),NaN,270.00,NaN
30,6.80,house,2160.0,Plot area 240(200.67 sq.m.),NaN,240.00,NaN
31,1.00,house,1215.0,Plot area 135(112.88 sq.m.),NaN,135.00,NaN
32,7.49,house,2700.0,Plot area 300(250.84 sq.m.),NaN,300.00,NaN
35,3.51,house,2844.0,Plot area 316(264.22 sq.m.),NaN,316.00,NaN
47,8.40,house,3240.0,Plot area 360(301.01 sq.m.),NaN,360.00,NaN


In [187]:
#converting sq yard to sq ft and sq m to sq ft
def convert_scale(row):

  if np.isnan(row['area']) or np.isnan(row['built_up_area']):
    return row['built_up_area']

  else:
    if round(row['area']/row['built_up_area'])==9.0:
      return row['built_up_area']*9

    elif round(row['area']/row['built_up_area'])==11.0:
      return row['built_up_area']*10.7

    else:
         return row['built_up_area']

In [188]:
all_nan_df['built_up_area']=all_nan_df.apply(convert_scale,axis=1)

In [189]:
all_nan_df

,price,property_type,area,areaWithType,super_built_up_area,built_up_area,carpet_area
0,7.25,house,2475.0,Plot area 275(229.94 sq.m.),NaN,2475.00,NaN
8,3.25,house,1512.0,Plot area 168(140.47 sq.m.),NaN,1512.00,NaN
11,8.48,house,3600.0,Plot area 400(334.45 sq.m.),NaN,3600.00,NaN
16,NaN,house,NaN,Plot area 670(560.21 sq.m.),NaN,670.00,NaN
29,0.34,house,270.0,Plot area 270(25.08 sq.m.),NaN,270.00,NaN
30,6.80,house,2160.0,Plot area 240(200.67 sq.m.),NaN,2160.00,NaN
31,1.00,house,1215.0,Plot area 135(112.88 sq.m.),NaN,1215.00,NaN
32,7.49,house,2700.0,Plot area 300(250.84 sq.m.),NaN,2700.00,NaN
35,3.51,house,2844.0,Plot area 316(264.22 sq.m.),NaN,2844.00,NaN
47,8.40,house,3240.0,Plot area 360(301.01 sq.m.),NaN,3240.00,NaN


In [190]:
df.update(all_nan_df)

In [191]:
df.isnull().sum()

,0
property_type,1
society,470
sector,0
price,19
price_per_sqft,19
area,19
areaWithType,1
bedRoom,1
bathroom,1
balcony,1


In [192]:
df.head()

,property_type,society,sector,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,floorNum,facing,agePossession,nearbyLocations,furnishDetails,features,super_built_up_area,built_up_area,carpet_area
0,house,NaN,sector 41,7.25,29293.0,2475.0,Plot area 275(229.94 sq.m.),9.0,8.0,3,servant room,4.0,South-West,10+ Year Old,"['Huda city centre metro station', 'Axis bank ATM', 'Hdfc ATM', 'Axis bank ATM', 'Citi bank ATM', 'Icici ATM', 'Punjab national bank ATM', 'Hdfc bank ATM', 'Standard chartered ATM', 'Centre For Sight Gurgaon Sector 29', 'Dr. Naval Mendiratta', 'Fortis Memorial Research Institute Fortis Vivekanand Hospital', 'Dr. Kutbuddin Akbary', 'Max Hospital', 'Shivam Hospital Gurgaon', 'Gardian Pharmacy', 'City Medical', 'Bharat petroleum', 'Hdfc bank', 'Icici bank', 'Beer & Whisky Bar', 'MoB Ministory of Beer', ""McDonald's"", 'Park Baluchi', ""Domino's Pizza"", 'Zura', 'The Oriental Bloom Chinese and Thai', 'Gola Sizzles', ""Hops 'N' Brew"", 'Pizza Hut', 'Spaghetti Kitchen & Bar', 'Bikanervala', 'Tocpao', 'PWO house', 'Black Mambaa', 'Gravity Space Bar', 'Roots', 'KFC', 'Ardor 29', 'ADDA', 'Gung the palace Korean restaurant', 'Walking Street', 'distillery', 'Swagath', 'Dhabba', 'Pizza Hut', 'Balaji Vegetarian Paradise', 'School of Inspired Leadership SOIL', 'Ncr library']","['5 Fan', '1 Exhaust Fan', '8 Geyser', '12 Light', '4 AC', '2 TV', '1 Modular Kitchen', '1 Curtains', '2 Bed', '2 Wardrobe', '1 Washing Machine', 'No Chimney', 'No Dining Table', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No Water Purifier']","['Security / Fire Alarm', 'Private Garden / Terrace', 'Water purifier', 'Maintenance Staff', 'Water Storage', 'No open drainage around', 'Visitor Parking', 'Park', 'Security Personnel', 'Natural Light', 'Internet/wi-fi connectivity', 'Airy Rooms', 'Spacious Interiors', 'Waste Disposal', 'Club house / Community Center']",NaN,2475.00,NaN
1,flat,vipul lavanya,sector 81,1.28,7174.0,1784.0,Carpet area: 1784 (165.74 sq.m.),3.0,3.0,3,"pooja room,servant room",1.0,North-East,5 to 10 Year Old,"['Yadav Clinic', 'Bangali Clinic', 'Dr. J. S. Sarkar Clinic', 'Orris Community Center', 'HP Petrol Pump', 'Vijay Petrol Pump', 'Petrol Pump', 'Essar Petrol Pump', 'Petrol pump Maitri motors', 'Petrol Pump Indian Oil', 'Indian Oil', ""McDonald's"", 'KFC', 'Pizza Hut', 'Rao Dhaba', ""McDonald's""]","['1 Bed', '3 Wardrobe', '7 Fan', '1 Exhaust Fan', '2 Geyser', '12 Light', '1 Modular Kitchen', '1 Chimney', '1 Curtains', 'No AC', 'No Dining Table', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Washing Machine', 'No Water Purifier']","['Feng Shui / Vaastu Compliant', 'Security / Fire Alarm', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Water Storage', 'Park', 'Visitor Parking']",NaN,NaN,1784.00
2,flat,m3m woodshire,sector 107,1.40,5929.0,2361.0,Super Built up area 2361(219.34 sq.m.),3.0,4.0,N,not available,1.0,East,0 to 1 Year Old,NaN,"['1 Light', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Modular Kitchen', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']","['Power Back-up', 'Intercom Facility', 'Lift(s)', 'Swimming Pool', 'Maintenance Staff', 'Park', 'Security Personnel', 'Internet/wi-fi connectivity', 'Shopping Centre', 'Fitness Centre / GYM', 'Club house / Community Center', 'Rain Water Harvesting']",2361.0,NaN,NaN
3,flat,ild greens,sector 37c,0.78,5714.0,1365.0,Super Built up area 1365(126.81 sq.m.)Built Up area: 1068.1 sq.ft. (99.23 sq.m.)Carpet area: 900 sq.ft. (83.61 sq.m.),2.0,2.0,3,not available,1.0,North,5 to 10 Year Old,"['The Esplanade Mall', 'Gurgaon Road', 'Delhi Jaipur Expressway', 'Green Field Public School', 'Sunrise University', 'K.D. Hospital', 'Indira Gandhi International Airport', 'Gurgaon Railway Station', 'infinity Business Park']","['2 Wardrobe', '3 Fan', '1 Exhaust Fan', '1 Geyser', '10 Light', '1 Modular Kitchen',

# 2.**additionalRoom**

In [193]:
df['additionalRoom'].value_counts()

,count
additionalRoom,
not available,1576
servant room,701
study room,249
others,225
pooja room,162
"study room,servant room",99
store room,99
"pooja room,servant room",79
"pooja room,study room,servant room,store room",69


In [194]:
#additional room
#list of new columns to be created
new_cols=['study room','servant room','store room','pooja room','others']

In [195]:
#populate the new columns based on additionalRoom column
for col in new_cols:
    df[col] = df['additionalRoom'].apply(lambda x: 1 if isinstance(x, (list, str)) and col in x else 0)


In [196]:
df[['additionalRoom','study room','servant room','store room','pooja room','others']].sample(5)

,additionalRoom,study room,servant room,store room,pooja room,others
763,"study room,servant room",1,1,0,0,0
551,not available,0,0,0,0,0
3732,"servant room,others",0,1,0,0,1
1875,"pooja room,study room,servant room",1,1,0,1,0
3239,not available,0,0,0,0,0


In [197]:
df.head()

,property_type,society,sector,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,floorNum,facing,agePossession,nearbyLocations,furnishDetails,features,super_built_up_area,built_up_area,carpet_area,study room,servant room,store room,pooja room,others
0,house,NaN,sector 41,7.25,29293.0,2475.0,Plot area 275(229.94 sq.m.),9.0,8.0,3,servant room,4.0,South-West,10+ Year Old,"['Huda city centre metro station', 'Axis bank ATM', 'Hdfc ATM', 'Axis bank ATM', 'Citi bank ATM', 'Icici ATM', 'Punjab national bank ATM', 'Hdfc bank ATM', 'Standard chartered ATM', 'Centre For Sight Gurgaon Sector 29', 'Dr. Naval Mendiratta', 'Fortis Memorial Research Institute Fortis Vivekanand Hospital', 'Dr. Kutbuddin Akbary', 'Max Hospital', 'Shivam Hospital Gurgaon', 'Gardian Pharmacy', 'City Medical', 'Bharat petroleum', 'Hdfc bank', 'Icici bank', 'Beer & Whisky Bar', 'MoB Ministory of Beer', ""McDonald's"", 'Park Baluchi', ""Domino's Pizza"", 'Zura', 'The Oriental Bloom Chinese and Thai', 'Gola Sizzles', ""Hops 'N' Brew"", 'Pizza Hut', 'Spaghetti Kitchen & Bar', 'Bikanervala', 'Tocpao', 'PWO house', 'Black Mambaa', 'Gravity Space Bar', 'Roots', 'KFC', 'Ardor 29', 'ADDA', 'Gung the palace Korean restaurant', 'Walking Street', 'distillery', 'Swagath', 'Dhabba', 'Pizza Hut', 'Balaji Vegetarian Paradise', 'School of Inspired Leadership SOIL', 'Ncr library']","['5 Fan', '1 Exhaust Fan', '8 Geyser', '12 Light', '4 AC', '2 TV', '1 Modular Kitchen', '1 Curtains', '2 Bed', '2 Wardrobe', '1 Washing Machine', 'No Chimney', 'No Dining Table', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No Water Purifier']","['Security / Fire Alarm', 'Private Garden / Terrace', 'Water purifier', 'Maintenance Staff', 'Water Storage', 'No open drainage around', 'Visitor Parking', 'Park', 'Security Personnel', 'Natural Light', 'Internet/wi-fi connectivity', 'Airy Rooms', 'Spacious Interiors', 'Waste Disposal', 'Club house / Community Center']",NaN,2475.00,NaN,0,1,0,0,0
1,flat,vipul lavanya,sector 81,1.28,7174.0,1784.0,Carpet area: 1784 (165.74 sq.m.),3.0,3.0,3,"pooja room,servant room",1.0,North-East,5 to 10 Year Old,"['Yadav Clinic', 'Bangali Clinic', 'Dr. J. S. Sarkar Clinic', 'Orris Community Center', 'HP Petrol Pump', 'Vijay Petrol Pump', 'Petrol Pump', 'Essar Petrol Pump', 'Petrol pump Maitri motors', 'Petrol Pump Indian Oil', 'Indian Oil', ""McDonald's"", 'KFC', 'Pizza Hut', 'Rao Dhaba', ""McDonald's""]","['1 Bed', '3 Wardrobe', '7 Fan', '1 Exhaust Fan', '2 Geyser', '12 Light', '1 Modular Kitchen', '1 Chimney', '1 Curtains', 'No AC', 'No Dining Table', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Washing Machine', 'No Water Purifier']","['Feng Shui / Vaastu Compliant', 'Security / Fire Alarm', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Water Storage', 'Park', 'Visitor Parking']",NaN,NaN,1784.00,0,1,0,1,0
2,flat,m3m woodshire,sector 107,1.40,5929.0,2361.0,Super Built up area 2361(219.34 sq.m.),3.0,4.0,N,not available,1.0,East,0 to 1 Year Old,NaN,"['1 Light', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Modular Kitchen', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']","['Power Back-up', 'Intercom Facility', 'Lift(s)', 'Swimming Pool', 'Maintenance Staff', 'Park', 'Security Personnel', 'Internet/wi-fi connectivity', 'Shopping Centre', 'Fitness Centre / GYM', 'Club house / Community Center', 'Rain Water Harvesting']",2361.0,NaN,NaN,0,0,0,0,0
3,flat,ild greens,sector 37c,0.78,5714.0,1365.0,Super Built up area 1365(126.81 sq.m.)Built Up area: 1068.1 sq.ft. (99.23 sq.m.)Carpet area: 900 sq.ft. (83.61 sq.m.),2.0,2.0,3,not available,1.0,North,5 to 10 Year Old,"['The Esplanade Mall', 'Gurgaon Road', 'Delhi Jaipur Expressway', 'Green Field Public School', 'Sunrise University', 'K.D. Hospital', 'Indira Gandhi International Airport', 'Gurgaon Railway Station', 'infinity Business Park']","['

#**3. agePossession**

In [198]:
df['agePossession'].value_counts()

,count
agePossession,
1 to 5 Year Old,1663
5 to 10 Year Old,571
0 to 1 Year Old,527
undefined,331
10+ Year Old,301
Under Construction,88
Within 6 months,69
Within 3 months,26
2023-12-01 00:00:00,22


In [199]:
def categorize_age_possession(value):
  if pd.isna(value):
    return "Undefined"

  if "0 to 1 Year Old" in value or "Within 6 months" in value or "Within 3 months" in value:
    return "New Property"

  if "1 to 5 Year Old" in value:
    return "Relatively New"

  if "5 to 10 Year Old" in value:
    return "Moderately Old"

  if "10+ Year Old" in value:
    return "Old Property"

  if "Under Construction" in value or "By" in value:
    return "Under Construction"

  try:
    a=int(value.split(" ")[0])
    if a>=2025:
     return "Under Construction"
    elif a<2025:
      return "Relatively New"

  except:
    return "Undefined"






In [200]:
df['agePossession']=df['agePossession'].apply(categorize_age_possession)

In [201]:
df['agePossession'].value_counts()

,count
agePossession,
Relatively New,1663
New Property,622
Moderately Old,571
Undefined,484
Old Property,301
Under Construction,130


In [202]:
df.head()

,property_type,society,sector,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,floorNum,facing,agePossession,nearbyLocations,furnishDetails,features,super_built_up_area,built_up_area,carpet_area,study room,servant room,store room,pooja room,others
0,house,NaN,sector 41,7.25,29293.0,2475.0,Plot area 275(229.94 sq.m.),9.0,8.0,3,servant room,4.0,South-West,Old Property,"['Huda city centre metro station', 'Axis bank ATM', 'Hdfc ATM', 'Axis bank ATM', 'Citi bank ATM', 'Icici ATM', 'Punjab national bank ATM', 'Hdfc bank ATM', 'Standard chartered ATM', 'Centre For Sight Gurgaon Sector 29', 'Dr. Naval Mendiratta', 'Fortis Memorial Research Institute Fortis Vivekanand Hospital', 'Dr. Kutbuddin Akbary', 'Max Hospital', 'Shivam Hospital Gurgaon', 'Gardian Pharmacy', 'City Medical', 'Bharat petroleum', 'Hdfc bank', 'Icici bank', 'Beer & Whisky Bar', 'MoB Ministory of Beer', ""McDonald's"", 'Park Baluchi', ""Domino's Pizza"", 'Zura', 'The Oriental Bloom Chinese and Thai', 'Gola Sizzles', ""Hops 'N' Brew"", 'Pizza Hut', 'Spaghetti Kitchen & Bar', 'Bikanervala', 'Tocpao', 'PWO house', 'Black Mambaa', 'Gravity Space Bar', 'Roots', 'KFC', 'Ardor 29', 'ADDA', 'Gung the palace Korean restaurant', 'Walking Street', 'distillery', 'Swagath', 'Dhabba', 'Pizza Hut', 'Balaji Vegetarian Paradise', 'School of Inspired Leadership SOIL', 'Ncr library']","['5 Fan', '1 Exhaust Fan', '8 Geyser', '12 Light', '4 AC', '2 TV', '1 Modular Kitchen', '1 Curtains', '2 Bed', '2 Wardrobe', '1 Washing Machine', 'No Chimney', 'No Dining Table', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No Water Purifier']","['Security / Fire Alarm', 'Private Garden / Terrace', 'Water purifier', 'Maintenance Staff', 'Water Storage', 'No open drainage around', 'Visitor Parking', 'Park', 'Security Personnel', 'Natural Light', 'Internet/wi-fi connectivity', 'Airy Rooms', 'Spacious Interiors', 'Waste Disposal', 'Club house / Community Center']",NaN,2475.00,NaN,0,1,0,0,0
1,flat,vipul lavanya,sector 81,1.28,7174.0,1784.0,Carpet area: 1784 (165.74 sq.m.),3.0,3.0,3,"pooja room,servant room",1.0,North-East,Moderately Old,"['Yadav Clinic', 'Bangali Clinic', 'Dr. J. S. Sarkar Clinic', 'Orris Community Center', 'HP Petrol Pump', 'Vijay Petrol Pump', 'Petrol Pump', 'Essar Petrol Pump', 'Petrol pump Maitri motors', 'Petrol Pump Indian Oil', 'Indian Oil', ""McDonald's"", 'KFC', 'Pizza Hut', 'Rao Dhaba', ""McDonald's""]","['1 Bed', '3 Wardrobe', '7 Fan', '1 Exhaust Fan', '2 Geyser', '12 Light', '1 Modular Kitchen', '1 Chimney', '1 Curtains', 'No AC', 'No Dining Table', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Washing Machine', 'No Water Purifier']","['Feng Shui / Vaastu Compliant', 'Security / Fire Alarm', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Water Storage', 'Park', 'Visitor Parking']",NaN,NaN,1784.00,0,1,0,1,0
2,flat,m3m woodshire,sector 107,1.40,5929.0,2361.0,Super Built up area 2361(219.34 sq.m.),3.0,4.0,N,not available,1.0,East,New Property,NaN,"['1 Light', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Modular Kitchen', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']","['Power Back-up', 'Intercom Facility', 'Lift(s)', 'Swimming Pool', 'Maintenance Staff', 'Park', 'Security Personnel', 'Internet/wi-fi connectivity', 'Shopping Centre', 'Fitness Centre / GYM', 'Club house / Community Center', 'Rain Water Harvesting']",2361.0,NaN,NaN,0,0,0,0,0
3,flat,ild greens,sector 37c,0.78,5714.0,1365.0,Super Built up area 1365(126.81 sq.m.)Built Up area: 1068.1 sq.ft. (99.23 sq.m.)Carpet area: 900 sq.ft. (83.61 sq.m.),2.0,2.0,3,not available,1.0,North,Moderately Old,"['The Esplanade Mall', 'Gurgaon Road', 'Delhi Jaipur Expressway', 'Green Field Public School', 'Sunrise University', 'K.D. Hospital', 'Indira Gandhi International Airport', 'Gurgaon Railway Station', 'infinity Business Park']","['2 Wardr

# **4.furnishDetails**

In [203]:
df[['furnishDetails','features']].sample(5)

,furnishDetails,features
2585,NaN,"['Security / Fire Alarm', 'Power Back-up', 'Private Garden / Terrace', 'Intercom Facility', 'Lift(s)', 'High Ceiling Height', 'Maintenance Staff', 'Water Storage', 'No open drainage around', 'Piped-gas', 'Visitor Parking', 'Swimming Pool', 'Park', 'Security Personnel', 'Internet/wi-fi connectivity', 'Low Density Society', 'Shopping Centre', 'Fitness Centre / GYM', 'Waste Disposal', 'Rain Water Harvesting', 'Club house / Community Center', 'Water softening plant']"
1191,"['40 Light', '1 Modular Kitchen', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']","['Park', 'Visitor Parking', 'Low Density Society']"
1069,"['1 Water Purifier', '1 Fridge', '20 Fan', '1 Exhaust Fan', '1 Dining Table', '20 Geyser', '1 Stove', '1 Curtains', '20 Bed', '20 Wardrobe', '1 Sofa', '1 Microwave', '1 Washing Machine', 'No AC', 'No Chimney', 'No Modular Kitchen', 'No Light', 'No TV']","['Water Storage', 'Park', 'Visitor Parking', 'Waste Disposal']"
482,"['1 Light', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Modular Kitchen', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']",NaN
2769,"['4 Wardrobe', '6 Fan', '2 Geyser', '6 Light', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Modular Kitchen', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Washing Machine', 'No Water Purifier']","['Security / Fire Alarm', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Water Storage', 'Park', 'Visitor Parking']"


In [204]:
#extract all unique furnishings from the furnishDetails column
all_furnishings=[]
for detail in df['furnishDetails'].dropna():
  furnishings=detail.replace('[','').replace(']','').replace("'","").split(',')
  all_furnishings.extend(furnishings)
unique_furnishings=list(set(all_furnishings))

#define a function to extract the count of a furnishing from the furnishDetails
def get_furnishing_count(details,furnishing):
  if isinstance(details,str):
    if f"No{furnishing}" in details:
      return 0
    pattern=re.compile(f"(\d+){furnishing}")
    match=pattern.search(details)
    if match:
      return int(match.group(1))
    elif furnishing in details:
      return 1
  return 0



In [205]:
#simplify the furnishings list by removing "No" prefix and numbers
columns_to_include=[re.sub(r'No|\d+','',furnishing).strip() for furnishing in unique_furnishings]
columns_to_include=list(set(columns_to_include))#get unique furnishings
columns_to_include=[furnishing for furnishing in columns_to_include if furnishing!='' and furnishing!=' ']

#create new columns for each unique furnishing and populate with counts
for furnishing in columns_to_include:
  df[furnishing]=df['furnishDetails'].apply(lambda x:get_furnishing_count(x,furnishing))


#create the dataframe with required columns
furnishings_df=df[['furnishDetails']+columns_to_include]

In [206]:
furnishings_df.shape

(3771, 19)

In [207]:
furnishings_df.head()

,furnishDetails,Fridge,Microwave,Curtains,TV,Water Purifier,Sofa,AC,Modular Kitchen,Chimney,Dining Table,Fan,Washing Machine,Exhaust Fan,Light,Wardrobe,Stove,Geyser,Bed
0,"['5 Fan', '1 Exhaust Fan', '8 Geyser', '12 Light', '4 AC', '2 TV', '1 Modular Kitchen', '1 Curtains', '2 Bed', '2 Wardrobe', '1 Washing Machine', 'No Chimney', 'No Dining Table', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No Water Purifier']",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,"['1 Bed', '3 Wardrobe', '7 Fan', '1 Exhaust Fan', '2 Geyser', '12 Light', '1 Modular Kitchen', '1 Chimney', '1 Curtains', 'No AC', 'No Dining Table', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Washing Machine', 'No Water Purifier']",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,"['1 Light', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Modular Kitchen', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,"['2 Wardrobe', '3 Fan', '1 Exhaust Fan', '1 Geyser', '10 Light', '1 Modular Kitchen', '1 Chimney', 'No AC', 'No Bed', 'No Curtains', 'No Dining Table', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Washing Machine', 'No Water Purifier']",1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,[],0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [208]:
furnishings_df.drop('furnishDetails',axis=1,inplace=True)

/tmp/ipython-input-208-2053818740.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  furnishings_df.drop('furnishDetails',axis=1,inplace=True)


In [209]:
furnishings_df.head()

,Fridge,Microwave,Curtains,TV,Water Purifier,Sofa,AC,Modular Kitchen,Chimney,Dining Table,Fan,Washing Machine,Exhaust Fan,Light,Wardrobe,Stove,Geyser,Bed
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [210]:
#using kmeans clustering
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [211]:
scaler=StandardScaler()
scaled_data=scaler.fit_transform(furnishings_df)

In [212]:
n_cluster=3 # optimal number of cluster can be chosen through elbow method
kmeans=KMeans(n_clusters=n_cluster,random_state=42)
kmeans.fit(scaled_data)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


KMeans(n_clusters=3, random_state=42)

In [213]:
#predict the cluster assignment for each row
cluster_assignments=kmeans.predict(scaled_data)

In [214]:
df=df.iloc[:,:-18]

In [215]:
df['furnishing_type']=cluster_assignments

In [216]:
df['furnishing_type']
# 0--> unfurnished, 1--> semifurnished, 2--->furnished

,furnishing_type
0,0
1,0
2,0
3,0
4,1
5,1
6,0
7,1
8,0
9,1


#**5.Features**

In [217]:
df['features'].sample(5)

,features
1628,"['Feng Shui / Vaastu Compliant', 'Security / Fire Alarm', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Water Storage', 'Park', 'Visitor Parking']"
3487,"['Feng Shui / Vaastu Compliant', 'Security / Fire Alarm', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Water Storage', 'Park', 'Visitor Parking']"
1029,"['Centrally Air Conditioned', 'Water purifier', 'Security / Fire Alarm', 'Power Back-up', 'Feng Shui / Vaastu Compliant', 'Private Garden / Terrace', 'Intercom Facility', 'Lift(s)', 'High Ceiling Height', 'Maintenance Staff', 'False Ceiling Lighting', 'Water Storage', 'Separate entry for servant room', 'No open drainage around', 'Bank Attached Property', 'Piped-gas', 'Internet/wi-fi connectivity', 'Recently Renovated', 'Visitor Parking', 'Swimming Pool', 'Park', 'Security Personnel', 'Natural Light', 'Airy Rooms', 'Spacious Interiors', 'Low Density Society', 'Waste Disposal', 'Rain Water Harvesting', 'Water softening plant', 'Shopping Centre', 'Fitness Centre / GYM', 'Club house / Community Center']"
3208,"['Feng Shui / Vaastu Compliant', 'Private Garden / Terrace', 'Maintenance Staff', 'Water Storage', 'Visitor Parking', 'Waste Disposal', 'Rain Water Harvesting']"
2637,"['Feng Shui / Vaastu Compliant', 'Private Garden / Terrace', 'Maintenance Staff', 'Water Storage', 'Park', 'Visitor Parking', 'Waste Disposal', 'Rain Water Harvesting']"


In [218]:
df['features'].isnull().sum()

np.int64(632)

In [219]:
import pandas as pd
app_df=pd.read_excel('real_estate_data.xlsx')
app_df.head(2)

,PropertyName,PropertySubName,NearbyLocations,LocationAdvantages,Link,PriceDetails,TopFacilities
0,Smartworld One DXP,"2, 3, 4 BHK Apartment in Sector 113, Gurgaon","['Bajghera Road', 'Palam Vihar Halt', 'DPSG Palam Vihar', 'Park Hospital', 'Gurgaon Railway Station']","{'Bajghera Road': '800 Meter', 'Palam Vihar Halt': '2.5 KM', 'DPSG Palam Vihar': '3.1 KM', 'Park Hospital': '3.1 KM', 'Gurgaon Railway Station': '4.9 KM', 'The NorthCap University': '5.4 KM', 'Dwarka Expy': '1.2 KM', 'Hyatt Place Gurgaon Udyog Vihar': '7.7 KM', 'Dwarka Sector 21, Metro Station': '7.2 KM', 'Pacific D21 Mall': '7.4 KM', 'Indira Gandhi International Airport': '14.7 KM', 'Hamoni Golf Camp': '6.2 KM', 'Fun N Food Waterpark': '8.8 KM', 'Accenture DDC5': '9 KM'}",https://www.99acres.com/smartworld-one-dxp-sector-113-gurgaon-npxid-r400415,"{'2 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '1,370 sq.ft.', 'price-range': '₹ 2 - 2.4 Cr'}, '3 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '1,850 - 2,050 sq.ft.', 'price-range': '₹ 2.25 - 3.59 Cr'}, '4 BHK': {'building_type': 'Apartment', 'area_type': 'Carpet Area', 'area': '2,600 sq.ft.', 'price-range': '₹ 3.24 - 4.56 Cr'}}","['Swimming Pool', 'Salon', 'Restaurant', 'Spa', 'Cafeteria', 'Sun Deck', '24x7 Security', 'Club House', 'Gated Community']"
1,M3M Crown,"3, 4 BHK Apartment in Sector 111, Gurgaon","['DPSG Palam Vihar Gurugram', 'The NorthCap University', 'Park Hospital, Palam Vihar', 'Pacific D21 Mall', 'Palam Vihar Halt Railway Station']","{'DPSG Palam Vihar Gurugram': '1.4 Km', 'The NorthCap University': '4.4 Km', 'Park Hospital, Palam Vihar': '1.4 Km', 'Pacific D21 Mall': '8.2 Km', 'Palam Vihar Halt Railway Station': '1.2 Km', 'Dwarka Sector 21 Metro Station': '8.1 Km', 'Dwarka Expressway': '450 m', 'Fun N Food Water Park': '8.1 Km', 'Indira Gandhi International Airport': '14.1 Km', 'Tau DeviLal Sports Complex': '11.2 Km', 'Hamoni Golf Camp': '5 Km', 'Hyatt Place': '6.1 Km', 'Altrade Business Centre': '11.2 Km'}",https://www.99acres.com/m3m-crown-sector-111-gurgaon-npxid-r404068,"{'3 BHK': {'building_type': 'Apartment', 'area_type': 'Super Built-up Area', 'area': '1,605 - 2,170 sq.ft.', 'price-range': '₹ 2.2 - 3.03 Cr'}, '4 BHK': {'building_type': 'Apartment', 'area_type': 'Super Built-up Area', 'area': '2,248 - 2,670 sq.ft.', 'price-range': '₹ 3.08 - 3.73 Cr'}}","['Bowling Alley', 'Mini Theatre', 'Manicured Garden', 'Swimming Pool', 'Flower Garden', 'Reading Lounge', 'Golf Course', 'Barbecue', 'Sauna']"


In [220]:
app_df['PropertyName']=app_df['PropertyName'].str.lower()

In [221]:
temp_df=df[df['features'].isnull()]

In [222]:
temp_df.shape

(632, 26)

In [223]:
x=temp_df.merge(app_df,left_on='society',right_on='PropertyName',how='left')['TopFacilities']

In [224]:
temp_df.index

Index([   7,   16,   17,   20,   31,   41,   42,   53,   54,   56,
       ...
       3726, 3740, 3741, 3742, 3749, 3751, 3760, 3765, 3767, 3770],
      dtype='int64', length=632)

In [225]:
df.loc[temp_df.index,'features']=x.values

In [226]:
df['features'].isnull().sum()

np.int64(478)

In [227]:
from sklearn.preprocessing import MultiLabelBinarizer
import ast

In [228]:
#Convert the string representation of lists in the feature column to actual lists
df['features_list']=df['features'].apply(lambda x:ast.literal_eval(x) if pd.notnull(x) and x.startswith('[') else [])

#use multilabelBinarizer to convert the features list into a binary matrix
mlb=MultiLabelBinarizer()
features_binary_matrix=mlb.fit_transform(df['features_list'])

#create a new dataframe with the binary matrix
features_binary_df=pd.DataFrame(features_binary_matrix,columns=mlb.classes_)

In [229]:
features_binary_df.head()

,24/7 Power Backup,24/7 Water Supply,24x7 Security,ATM,Aerobics Centre,Air Hockey,Airy Rooms,Amphitheatre,Automated Car Wash,Badminton Court,Bank Attached Property,Banquet Hall,Bar/Chill-Out Lounge,Barbecue,Basketball Court,Beach Volley Ball Court,Billiards,Bowling Alley,Bus Shelter,Business Lounge,CCTV Camera Security,Cafeteria,Car Parking,Car wash area,Card Room,Centrally Air Conditioned,Changing Area,Children's Play Area,Cigar Lounge,Clinic,Club House,Club house / Community Center,Community Hall,Concierge Service,Conference room,Creche/Day care,Cricket Pitch,Doctor on Call,Earthquake Resistant,Entrance Lobby,False Ceiling Lighting,Feng Shui / Vaastu Compliant,Fire Fighting Systems,Fitness Centre / GYM,Flower Garden,Food Court,Foosball,Football,Fountain,Gated Community,Gazebo,Golf Course,Grocery Shop,Gymnasium,High Ceiling Height,High Speed Elevators,Infinity Pool,Intercom Facility,Internal Street Lights,Internet/wi-fi connectivity,Jacuzzi,Jogging Track,Landscape Garden,Laundry,Lawn Tennis Court,Library,Lift(s),Lounge,Low Density Society,Maintenance Staff,Manicured Garden,Medical Centre,Milk Booth,Mini Theatre,Multipurpose Court,Multipurpose Hall,Natural Light,Natural Pond,No open drainage around,Park,Party Lawn,Pergola,Piped Gas,Piped-gas,Pool Table,Power Back up Lift,Power Back-up,Private Garden / Terrace,Property Staff,RO System,Rain Water Harvesting,Reading Lounge,Recently Renovated,Reflexology Park,Restaurant,Salon,Sauna,School,Security / Fire Alarm,Security Personnel,Separate entry for servant room,Sewage Treatment Plant,Shopping Centre,Skating Rink,Solar Lighting,Solar Water Heating,Spa,Spacious Interiors,Squash Court,Steam Room,Sun Deck,Swimming Pool,Temple,Terrace Garden,Theatre,Toddler Pool,Valet Parking,Vastu Compliant,Video Door Security,Visitor Parking,Visitors Parking,Volley Ball Court,Waiting Lounge,Waste Disposal,Water Softener Plant,Water Storage,Water purifier,Water softening plant,Wi-Fi Connectivity,Yoga/Meditation Area
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0


In [230]:
features_binary_df.shape

(3771, 130)

In [231]:
#calculating luxury_score
weights = {
    '24/7 Power Backup': 8,
    '24/7 Water Supply': 4,
    '24x7 Security': 7,
    'ATM': 4,
    'Aerobics Centre': 6,
    'Airy Rooms': 8,
    'Amphitheatre': 7,
    'Badminton Court': 7,
    'Banquet Hall': 8,
    'Bar/Chill-Out Lounge': 9,
    'Barbecue': 7,
    'Basketball Court': 7,
    'Billiards': 7,
    'Bowling Alley': 8,
    'Business Lounge': 9,
    'CCTV Camera Security': 8,
    'Cafeteria': 6,
    'Car Parking': 6,
    'Card Room': 6,
    'Centrally Air Conditioned': 9,
    'Changing Area': 6,
     "Children's Play Area": 7,
    'Cigar Lounge': 9,
    'Clinic': 5,
    'Club House': 9,
    'Concierge Service': 9,
    'Conference room': 8,
    'Creche/Day care': 7,
    'Cricket Pitch': 7,
    'Doctor on Call': 6,
    'Earthquake Resistant': 5,
    'Entrance Lobby': 7,
    'False Ceiling Lighting': 6,
    'Feng Shui / Vaastu Compliant': 5,
    'Fire Fighting Systems': 8,
    'Fitness Centre / GYM': 8,
    'Flower Garden': 7,
    'Food Court': 6,
    'Foosball': 5,
    'Football': 7,
    'Fountain': 7,
    'Gated Community': 7,
    'Golf Course': 10,
    'Grocery Shop': 6,
    'Gymnasium': 8,
    'High Ceiling Height': 8,
    'High Speed Elevators': 8,
    'Infinity Pool': 9,
    'Intercom Facility': 7,
    'Internal Street Lights': 6,
    'Internet/wi-fi connectivity': 7,
    'Jacuzzi': 9,
    'Jogging Track': 7,
    'Landscape Garden': 8,
    'Laundry': 6,
    'Lawn Tennis Court': 8,
    'Library': 8,
    'Lounge': 8,
    'Low Density Society': 7,
    'Maintenance Staff': 6,
    'Manicured Garden': 7,
    'Medical Centre': 5,
    'Milk Booth': 4,
    'Mini Theatre': 9,
    'Multipurpose Court': 7,
    'Multipurpose Hall': 7,
    'Natural Light': 8,
    'Natural Pond': 7,
    'Park': 8,
    'Party Lawn': 8,
    'Piped Gas': 7,
    'Pool Table': 7,
    'Power Back up Lift': 8,
    'Private Garden / Terrace': 9,
    'Property Staff': 7,
     'RO System': 7,
    'Rain Water Harvesting': 7,
    'Reading Lounge': 8,
    'Restaurant': 8,
    'Salon': 8,
    'Sauna': 9,
    'Security / Fire Alarm': 9,
    'Security Personnel': 9,
    'Separate entry for servant room': 8,
    'Sewage Treatment Plant': 6,
    'Shopping Centre': 7,
    'Skating Rink': 7,
    'Solar Lighting': 6,
    'Solar Water Heating': 7,
    'Spa': 9,
    'Spacious Interiors': 9,
    'Squash Court': 8,
    'Steam Room': 9,
    'Sun Deck': 8,
    'Swimming Pool': 8,
    'Temple': 5,
    'Theatre': 9,
    'Toddler Pool': 7,
    'Valet Parking': 9,
    'Video Door Security': 9,
    'Visitor Parking': 7,
    'Water Softener Plant': 7,
     'Water Storage': 7,
    'Water purifier': 7,
    'Yoga/Meditation Area': 7
}

In [232]:
features_binary_df.head()

,24/7 Power Backup,24/7 Water Supply,24x7 Security,ATM,Aerobics Centre,Air Hockey,Airy Rooms,Amphitheatre,Automated Car Wash,Badminton Court,Bank Attached Property,Banquet Hall,Bar/Chill-Out Lounge,Barbecue,Basketball Court,Beach Volley Ball Court,Billiards,Bowling Alley,Bus Shelter,Business Lounge,CCTV Camera Security,Cafeteria,Car Parking,Car wash area,Card Room,Centrally Air Conditioned,Changing Area,Children's Play Area,Cigar Lounge,Clinic,Club House,Club house / Community Center,Community Hall,Concierge Service,Conference room,Creche/Day care,Cricket Pitch,Doctor on Call,Earthquake Resistant,Entrance Lobby,False Ceiling Lighting,Feng Shui / Vaastu Compliant,Fire Fighting Systems,Fitness Centre / GYM,Flower Garden,Food Court,Foosball,Football,Fountain,Gated Community,Gazebo,Golf Course,Grocery Shop,Gymnasium,High Ceiling Height,High Speed Elevators,Infinity Pool,Intercom Facility,Internal Street Lights,Internet/wi-fi connectivity,Jacuzzi,Jogging Track,Landscape Garden,Laundry,Lawn Tennis Court,Library,Lift(s),Lounge,Low Density Society,Maintenance Staff,Manicured Garden,Medical Centre,Milk Booth,Mini Theatre,Multipurpose Court,Multipurpose Hall,Natural Light,Natural Pond,No open drainage around,Park,Party Lawn,Pergola,Piped Gas,Piped-gas,Pool Table,Power Back up Lift,Power Back-up,Private Garden / Terrace,Property Staff,RO System,Rain Water Harvesting,Reading Lounge,Recently Renovated,Reflexology Park,Restaurant,Salon,Sauna,School,Security / Fire Alarm,Security Personnel,Separate entry for servant room,Sewage Treatment Plant,Shopping Centre,Skating Rink,Solar Lighting,Solar Water Heating,Spa,Spacious Interiors,Squash Court,Steam Room,Sun Deck,Swimming Pool,Temple,Terrace Garden,Theatre,Toddler Pool,Valet Parking,Vastu Compliant,Video Door Security,Visitor Parking,Visitors Parking,Volley Ball Court,Waiting Lounge,Waste Disposal,Water Softener Plant,Water Storage,Water purifier,Water softening plant,Wi-Fi Connectivity,Yoga/Meditation Area
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0


In [233]:
luxury_score=features_binary_df[list(weights.keys())].multiply(list(weights.values())).sum(axis=1)

In [234]:
df['luxury_score']=luxury_score

In [235]:
df.head()

,property_type,society,sector,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,additionalRoom,floorNum,facing,agePossession,nearbyLocations,furnishDetails,features,super_built_up_area,built_up_area,carpet_area,study room,servant room,store room,pooja room,others,furnishing_type,features_list,luxury_score
0,house,NaN,sector 41,7.25,29293.0,2475.0,Plot area 275(229.94 sq.m.),9.0,8.0,3,servant room,4.0,South-West,Old Property,"['Huda city centre metro station', 'Axis bank ATM', 'Hdfc ATM', 'Axis bank ATM', 'Citi bank ATM', 'Icici ATM', 'Punjab national bank ATM', 'Hdfc bank ATM', 'Standard chartered ATM', 'Centre For Sight Gurgaon Sector 29', 'Dr. Naval Mendiratta', 'Fortis Memorial Research Institute Fortis Vivekanand Hospital', 'Dr. Kutbuddin Akbary', 'Max Hospital', 'Shivam Hospital Gurgaon', 'Gardian Pharmacy', 'City Medical', 'Bharat petroleum', 'Hdfc bank', 'Icici bank', 'Beer & Whisky Bar', 'MoB Ministory of Beer', ""McDonald's"", 'Park Baluchi', ""Domino's Pizza"", 'Zura', 'The Oriental Bloom Chinese and Thai', 'Gola Sizzles', ""Hops 'N' Brew"", 'Pizza Hut', 'Spaghetti Kitchen & Bar', 'Bikanervala', 'Tocpao', 'PWO house', 'Black Mambaa', 'Gravity Space Bar', 'Roots', 'KFC', 'Ardor 29', 'ADDA', 'Gung the palace Korean restaurant', 'Walking Street', 'distillery', 'Swagath', 'Dhabba', 'Pizza Hut', 'Balaji Vegetarian Paradise', 'School of Inspired Leadership SOIL', 'Ncr library']","['5 Fan', '1 Exhaust Fan', '8 Geyser', '12 Light', '4 AC', '2 TV', '1 Modular Kitchen', '1 Curtains', '2 Bed', '2 Wardrobe', '1 Washing Machine', 'No Chimney', 'No Dining Table', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No Water Purifier']","['Security / Fire Alarm', 'Private Garden / Terrace', 'Water purifier', 'Maintenance Staff', 'Water Storage', 'No open drainage around', 'Visitor Parking', 'Park', 'Security Personnel', 'Natural Light', 'Internet/wi-fi connectivity', 'Airy Rooms', 'Spacious Interiors', 'Waste Disposal', 'Club house / Community Center']",NaN,2475.00,NaN,0,1,0,0,0,0,"[Security / Fire Alarm, Private Garden / Terrace, Water purifier, Maintenance Staff, Water Storage, No open drainage around, Visitor Parking, Park, Security Personnel, Natural Light, Internet/wi-fi connectivity, Airy Rooms, Spacious Interiors, Waste Disposal, Club house / Community Center]",94
1,flat,vipul lavanya,sector 81,1.28,7174.0,1784.0,Carpet area: 1784 (165.74 sq.m.),3.0,3.0,3,"pooja room,servant room",1.0,North-East,Moderately Old,"['Yadav Clinic', 'Bangali Clinic', 'Dr. J. S. Sarkar Clinic', 'Orris Community Center', 'HP Petrol Pump', 'Vijay Petrol Pump', 'Petrol Pump', 'Essar Petrol Pump', 'Petrol pump Maitri motors', 'Petrol Pump Indian Oil', 'Indian Oil', ""McDonald's"", 'KFC', 'Pizza Hut', 'Rao Dhaba', ""McDonald's""]","['1 Bed', '3 Wardrobe', '7 Fan', '1 Exhaust Fan', '2 Geyser', '12 Light', '1 Modular Kitchen', '1 Chimney', '1 Curtains', 'No AC', 'No Dining Table', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Washing Machine', 'No Water Purifier']","['Feng Shui / Vaastu Compliant', 'Security / Fire Alarm', 'Intercom Facility', 'Lift(s)', 'Maintenance Staff', 'Water Storage', 'Park', 'Visitor Parking']",NaN,NaN,1784.00,0,1,0,1,0,0,"[Feng Shui / Vaastu Compliant, Security / Fire Alarm, Intercom Facility, Lift(s), Maintenance Staff, Water Storage, Park, Visitor Parking]",49
2,flat,m3m woodshire,sector 107,1.40,5929.0,2361.0,Super Built up area 2361(219.34 sq.m.),3.0,4.0,N,not available,1.0,East,New Property,NaN,"['1 Light', 'No AC', 'No Bed', 'No Chimney', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Modular Kitchen', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Wardrobe', 'No Washing Machine', 'No Water Purifier']","['Power Back-up', 'Intercom Facility', 'Lift(s)', 'Swimming Pool', 'Maintenance Staff', 'Park', 'Security Personnel', 'Internet/wi-fi connectivity', 'Shopping Centre', 'Fitness Centre / GYM', 'Club house / Community Center', 'Rain Water Harve

In [236]:
df.drop(columns=['nearbyLocations','furnishDetails','features','features_list','additionalRoom'],inplace=True)

In [237]:
df.columns


Index(['property_type', 'society', 'sector', 'price', 'price_per_sqft', 'area',
       'areaWithType', 'bedRoom', 'bathroom', 'balcony', 'floorNum', 'facing',
       'agePossession', 'super_built_up_area', 'built_up_area', 'carpet_area',
       'study room', 'servant room', 'store room', 'pooja room', 'others',
       'furnishing_type', 'luxury_score'],
      dtype='object')

In [238]:
df.sample(1)

,property_type,society,sector,price,price_per_sqft,area,areaWithType,bedRoom,bathroom,balcony,floorNum,facing,agePossession,super_built_up_area,built_up_area,carpet_area,study room,servant room,store room,pooja room,others,furnishing_type,luxury_score
2239,house,NaN,sector 37d,2.5,13157.0,1900.0,Plot area 1900(176.52 sq.m.),5.0,5.0,3+,2.0,NaN,Relatively New,NaN,1900.0,NaN,0,0,0,0,0,1,0


In [239]:
df.shape

(3771, 23)

In [240]:
df.to_csv('gurgaon_properties_v3.csv',index=False)